# Punto 2 Búsqueda binaria usando índice invertido (BSII)
## Integrantes
* Juan Esteban Arboleda
* Luccas Rojas

### 1. Preprocesamiento
Lo primero que se llevara a cabo para poder hacer la busqueda binaria a travez de indice invertido es la tokenizacion de los documentos y generacion del vocabulario. Ademas es importante tener en cuenta que el vocabulario debe estar ordenado, no debe contener stop-words, debe estar stemizado y normalizado

* A continucion se cargan los documentos y los queries en una estructura de datos, se debe cambiar document_path y query_path por la ruta donde se encuentran los documentos y los queries respectivamente

In [24]:
import os
import pandas as pd
import numpy as np
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import time

# Rutas a definir segun la ubicacion de los archivos
DOCUMENTS_PATH = '../data/docs-raw-texts'
QUERIES_PATH = '../data/queries-raw-texts'
QUERIES_RESULTS_FILE_PATH = "../data/BSII-AND-queries_results.tsv"

def load_documents(folder_path: str) -> pd.DataFrame:
    """
    Returns a Pandas DataFrame where each row represents a document in folder_path.
    The DataFrame will have as many rows as there are documents in folder_path

        Parameters
        ----------
            folder_path: str
                The path to the folder that contains the documents to load
    
        Returns
        --------
            documents: pd.DataFrame
                Pandas DataFrame with two columns: "filename" and "body"
    """
    documents = []
    index = []
    id = 1
    columns = ['filename', 'body']
    for filename in os.listdir(folder_path):
        text = pd.read_xml(os.path.join(folder_path, filename))['raw'].tolist()[1]
        filtered_text = text.replace('\n', ' ').replace('\xa0', ' ')
        document = [filename, filtered_text]
        documents.append(document)
        index.append(id)
        id += 1

    return pd.DataFrame(documents, index, columns)

documents = load_documents(DOCUMENTS_PATH)
queries = load_documents(QUERIES_PATH)

documents

,filename,body
1,wes2015.d001.naf,William Beaumont and the Human Digestion. Wil...
2,wes2015.d002.naf,Selma Lagerlöf and the wonderful Adventures of...
3,wes2015.d003.naf,Ferdinand de Lesseps and the Suez Canal. Ferd...
4,wes2015.d004.naf,Walt Disney’s ‘Steamboat Willie’ and the Rise ...
5,wes2015.d005.naf,Eugene Wigner and the Structure of the Atomic ...
...,...,...
327,wes2015.d327.naf,James Parkinson and Parkinson’s Disease. Wood...
328,wes2015.d328.naf,Juan de la Cierva and the Autogiro. Demonstra...
329,wes2015.d329.naf,Squire Whipple – The Father of the Iron Bridge...
330,wes2015.d330.naf,William Playfair and the Beginnings of Infogra...


Primero que todo tokenizamos el texto, para esto utilizamos el word tokenize de la libreria NLTK

In [25]:
nltk.download('punkt')
nltk.download('stopwords')

documents['tokens'] = documents['body'].apply(word_tokenize)
queries['tokens'] = queries['body'].apply(word_tokenize)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luccas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\luccas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Removemos todos los signos de puntuacion, contracciones del ingles y dejamos el texto todo en minusculas (normalizar) 

In [26]:
def remove_punctuation(token_list):
    return [token.lower() for token in token_list if (token not in string.punctuation and (len(token)>1 or token.isnumeric()))]

documents['tokens']=documents['tokens'].apply(lambda x: remove_punctuation(x))
queries['tokens']=queries['tokens'].apply(lambda x: remove_punctuation(x))

Luego de tokenizar, dejar todo en minusculas, quitaremos las stop words para que reduzcan el vocabulario y no afecten el resultado final. Para esto usaremos la libreria nltk y su metodo stopwords.words('english').

In [27]:
stop_words = set(stopwords.words('english'))

#TODO no se si normalizar cuente como poner todo en minusculas
def remove_stop_words(token_list):
    return [token for token in token_list if token not in stop_words]

documents['tokens']=documents['tokens'].apply(lambda x: remove_stop_words(x))
queries['tokens']=queries['tokens'].apply(lambda x: remove_stop_words(x))

Luego de eliminar las stop words se hace stemming a las palabras restantes.

In [28]:
stemmer = PorterStemmer()
def stemming(token_list):
    return [stemmer.stem(token) for token in token_list]

documents['tokens']=documents['tokens'].apply(lambda x: stemming(x))
queries['tokens']=queries['tokens'].apply(lambda x: stemming(x))

En este punto el texto de cada documento y query esta en un formato mas facil de procesar, por lo que se procede a realizar la representacion vectorial de los documentos y queries.

## 2. Representación de los datos

A continuación se hace la implementación para transformar el anterior dataframe en una estructura de indice inertido para así poder realizar busquedas binarias 

In [29]:
def create_inverted_index(documents: pd.DataFrame) -> dict:
    """
    Creates the inverted index for a document set.

    Params
    ------
        documents: pd.DataFrame
            A Pandas DataFrame that represents the document set. The
            DataFrame should have the following columns: "filename", "body".
            DataFrame's index should correspond to the document id
        
    Returns
    -------
        inverted_index: dict
            A python dictionary that represents the inverted index.
            Keys are the terms in the vocabulary.
            Each value has a "df" (document frecuency) and "postings".
            "postings" are a numpy array
    """
    inverted_index = {}

    for id, document in documents.iterrows():
        for token in document['tokens']:
            if token not in inverted_index:
                inverted_index[token] = {"df": 0, "postings": []}
            if id not in inverted_index[token]["postings"]:
                inverted_index[token]["df"] += 1
                inverted_index[token]["postings"].append(id)
    
    return inverted_index

inverted_index = create_inverted_index(documents)

Como se pudo observar en el anterior código, se crea un diccionario que almacenara el índice invertido haciendo un recorrido por cada uno de los documentos y sus tokens. Agregando así todos los tokens del vocabulario y añadiendo a cada token el listado de documentos que contienen ese token. El vocabulario final cuenta con 14682 tokens.

## 3. Modelamiento

In [30]:
def and_intersect(postings1: list, postings2: list) -> list:
    """
    Returns the intersection of two postings lists
    """

    i = 0
    j = 0

    intersection = []

    # Merge algorith taken from the book
    while(i < len(postings1) and j < len(postings2)):
        docId1 = postings1[i]
        docId2 = postings2[j]
        if docId1 == docId2:
            intersection.append(docId1)
            i += 1
            j += 1
        elif docId1 < docId2:
            i += 1
        else:
            j += 1

    return intersection


def and_search(terms: list, inverted_index: dict) -> list:
    """
    Returns a list with the ids if the documents that
    contain all of the terms in terms list.

        Params
        ------
            terms: list[str]
                list of terms to look for in the documents
            
            inverted_index: dict
                Inverted index created from the document base
    """
    term_df_list = []

    for term in terms:
        if term in inverted_index:
            term_df_list.append({
                "term": term,
                "df": inverted_index[term]["df"]
            })
        else:
            # If a term that is not in the inverted index
            # is found. That means that there is no document
            # in the document base that meets the query.
            # Hence, an empty array is returned
            return []
        
    # If there is only one term to match, the function
    # returns the postings of that term
    if len(term_df_list) == 1:
        return inverted_index[term_df_list[0]["term"]]["postings"]

    # Sort term_df_list based on df
    term_df_list.sort(key=lambda elem: elem["df"])

    # Initialize intersection as the smallest postings list
    intersection = inverted_index[term_df_list[0]["term"]]["postings"]

    for i in range(1, len(term_df_list)):
        # If there are no items in the current intersection
        # there is no point in calculating the intersection
        # for the rest of the postings.
        # Hence, the function returns current (empty) intersection
        if len(intersection) == 0:
            return intersection
        
        postings_i = inverted_index[term_df_list[i]["term"]]["postings"]

        # calculate the intersection of current intersection with the next
        # smallest posting list
        intersection = and_intersect(intersection, postings_i)

    return intersection

# query processing

# Clear output file contents
open(QUERIES_RESULTS_FILE_PATH, "w").close()

# Loop through queries
for i, query in queries.iterrows():
    # Open output file
    file = open(QUERIES_RESULTS_FILE_PATH, "a")
    query_str = query['filename'].replace('.naf', '').replace('wes2015.', '')
    file.write(query_str + " ")

    # Perform AND query with all the terms in the query
    res = and_search(query["tokens"], inverted_index)

    # Write output file
    for docId in res:
        if docId < 10:
            doc_str = "d00" + str(docId)
        elif docId < 100:
            doc_str = "d0" + str(docId)
        else:
            doc_str = "d" + str(docId)
        file.write(doc_str)
        if docId != res[len(res) - 1]:
            file.write(",")
    if query_str != "q46":
        file.write("\n")

file.close()